In [10]:
import os.path
import natsort
import random
import itertools
import numpy as np
import cv2

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"]=15,15

BORDER = 25
SIZE = 250
N_NEWIMAGES = 5

In [3]:
def main():
    images = {}
    dir_src = input("Path para imagem ou diretório contendo imagens: ")

    if os.path.isfile(dir_src):
        grupo = input("Digite o nome do grupo ao qual a imagem pertence: ")
        images[grupo] = [dir_src]
    elif os.path.isdir(dir_src):
        for path, subdirs, files in os.walk(dir_src):
            createNewGroup = True
            for name in files:
                if name.lower().endswith(".jpg"):
                    if createNewGroup:
                        images[path.split("\\")[-1]] = []
                        createNewGroup = False
                    #end if
                    images[path.split("\\")[-1]].append(os.path.join(path, name))
                #end if
            #end for
        #end for
    #end if

    if not os.path.isdir("training_images"):
        os.mkdir("training_images")
    #end if
    
    cropAndResizeImgs(images)
#end def

In [4]:
def getThresholdForImages():
    thresholdRed = 255
    thresholdGreen = 255
    thersholdBlue = 255

    while True:
        thresholdRed = int(input("Digite um valor para o thresholdRed entre 0 e 255: "))
        if 0 <= thresholdRed <= 255:
            break
        #end if
    #end while

    while True:
        thresholdGreen = int(input("Digite um valor para o thresholdGreen entre 0 e 255: "))
        if 0 <= thresholdGreen <= 255:
            break
        #end if
    #end while

    while True:
        thresholdBlue = int(input("Digite um valor para o thresholdBlue entre 0 e 255: "))
        if 0 <= thresholdBlue <= 255:
            break
        #end if
    #end while
        
    threshold = [thresholdBlue, thresholdGreen,thresholdRed]
    return threshold
#end def

In [5]:
def cropImageObject(img_threshold, img):
    img_Gray = cv2.cvtColor(img_threshold, cv2.COLOR_BGR2GRAY)
    x,y,w,h = cv2.boundingRect(img_Gray)

    #imgplot = plt.imshow(img_Gray)
            
    if h > w:
        if h % 2 == 0:
            img_Crop = img[y:y+h, x-(h-w)//2:x+w+(h-w)//2]
        else:
            img_Crop = img[y:y+h, x-(h-w)//2:x+w+(h-w)//2+1]
        #end if
    elif h < w:
        if h % 2 == 0:
            img_Crop = img[y-(w-h)//2:y+h+(w-h)//2, x:x+w]
        else:
            img_Crop = img[y-(w-h)//2:y+h+(w-h)//2+1, x:x+w]
        #end if
    elif h == 2:
        img_Crop = img[y:y+h, x:x+w]
    #end if
        
    return img_Crop
#end def

In [6]:
def cropAndResizeImgs(images):
    first_loop = True
    for group in images:
        imgNumber = 0
        
        if not os.path.isdir("training_images\\%s"%group):
            os.mkdir("training_images\\%s"%group)
        else:
            files = os.listdir("training_images\\%s"%group)
            files_jpg = [i for i in files if i.lower().endswith('.jpg')]
            if len(files_jpg) == 0:
                imgNumber = 0
            else:
                imgNumber = int(natsort.natsorted(files_jpg,reverse=False)[-1].split("-")[-1].split(".")[0]) + 1
         #end if

        for img in images[group]:
            try:
                img_BGR = np.array(cv2.imread(img))

                #imgplot = plt.imshow(cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB))

                lowerBound = np.array([0, 0, 0])
                if first_loop:
                    upperBound = np.array(getThresholdForImages())
                    first_loop = False
                #end if

                mask = cv2.inRange(img_BGR, lowerBound, upperBound)
                img_threshold = cv2.bitwise_and(img_BGR,img_BGR, mask= mask)

                #imgplot = plt.imshow(cv2.cvtColor(img_threshold, cv2.COLOR_BGR2RGB))  

                img_Crop = cropImageObject(img_threshold,img_BGR)

                #imgplot = plt.imshow(img_Crop)

                img_resized = cv2.resize(img_Crop, (SIZE,SIZE))
                #imgplot = plt.imshow(img_resized)

                img_Border10 = cv2.copyMakeBorder(img_resized, BORDER, BORDER, BORDER, BORDER, cv2.BORDER_CONSTANT, value=[255,255,255] )
                #imgplot = plt.imshow(cv2.cvtColor(img_Border10,cv2.COLOR_BGR2RGB))

                img_name = group + "-" + str(imgNumber)
                cv2.imwrite(("training_images\\%s\\%s.jpg"%(group,img_name)),img_Border10)
                
                imgNumber += 1

            except Exception as e:
                print(("ERROR: IMAGEM %s: "%img),str(e))
        #end for
    #end for
#end def

In [7]:
def main2():
    images = {}
    dir_src = input("Path com imagem(s) bases: ")

    if os.path.isfile(dir_src):
        grupo = input("Digite o nome do grupo ao qual a imagem pertence: ")
        images[grupo] = [dir_src]
    elif os.path.isdir(dir_src):
        for path, subdirs, files in os.walk(dir_src):
            createNewGroup = True
            for name in files:
                if name.lower().endswith(".jpg"):
                    if createNewGroup:
                        images[path.split("\\")[-1]] = []
                        createNewGroup = False
                    #end if
                    images[path.split("\\")[-1]].append(os.path.join(path, name))
                #end if
            #end for
        #end for
    #end if

    if not os.path.isdir("training_images"):
        os.mkdir("training_images")
    
    generateImages(images,N_NEWIMAGES)
    #end if
#end def

In [18]:
def generateImages(images,numberImgs):
    for group in images:
        
        imgNumber = 0
        
        if not os.path.isdir("training_images\\%s"%group):
            os.mkdir("training_images\\%s"%group)
        else:
            files = os.listdir("training_images\\%s"%group)
            files_jpg = [i for i in files if i.lower().endswith('.jpg')]
            if len(files_jpg) == 0:
                imgNumber = 0
            else:
                imgNumber = int(natsort.natsorted(files_jpg,reverse=False)[-1].split("-")[-1].split(".")[0]) + 1
         #end if
        
        for img in images[group]:
            img_BGR = np.array(cv2.imread(img))
            img_Crop = img_BGR[BORDER-1:BORDER+SIZE-1, BORDER-1:BORDER+SIZE-1]
            
            # Generate all possible non-repeating pairs
            numbers = [i for i in range(0,51)]
            pairs = np.array(list(itertools.combinations(numbers, 2)))
            random.shuffle(pairs)
            
            for i in range(0,numberImgs):
                [x,y] = random.choice(pairs)
                np.delete(pairs,[x,y])
                
                left = x if x <= 24 else 50 - x
                right = 50 - x if x <= 24 else x
                top = y if y <= 24 else 50 - y
                bottom = 50 - y if y <= 24 else y

                newImage = cv2.copyMakeBorder(img_Crop, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[255,255,255] )
                
                img_name = group + "-" + str(imgNumber)
                cv2.imwrite(("training_images\\%s\\%s.jpg"%(group,img_name)),newImage)
                
                imgNumber += 1
        #end for
    #end for
#end def    
    

In [19]:
#main()
main2()
print("Fim")

Path com imagem(s) bases: C:\Users\Adria\TCC\GitHub\TCC_CNN\training_images - 300x300
Fim
